In [4]:
%load_ext autoreload
# from google.colab import drive
# drive.mount('/content/drive')
#%cd "/content/drive/MyDrive/ldh"
# import nltk
# nltk.download('punkt')

%cd "/Users/danielpham/Google Drive/ldh"
%pip install transformers datasets pytorch-lightning nltk matplotlib
%pylab inline

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload
/Users/danielpham/Google Drive/ldh
     |████████████████████████████████| 8.5 MB 7.1 MB/s 
     |████████████████████████████████| 2.2 MB 36.4 MB/s 
     |████████████████████████████████| 61 kB 234 kB/s 
You should consider upgrading via the '/Users/danielpham/.pyenv/versions/3.8.7/bin/python -m pip install --upgrade pip' command.
Note: you may need to restart the kernel to use updated packages.
Populating the interactive namespace from numpy and matplotlib


## Data Preparation
- Data is loaded through LDHDataModule. 
- Returned 

In [7]:
%autoreload 2 
from src.LDHDataModule import LDHDataModule
from src.LightningReapp import LightningReapp, ReappTrainer

ldhdata = LDHDataModule(batch_size=16, strat='far', kfolds=3)

  3%|▎         | 357/13470 [00:00<00:03, 3567.56ex/s]Regenerating training data.
Saving training dataset to /Users/danielpham/Google Drive/ldh/output/training
Encoding Train Data:
100%|██████████| 13470/13470 [00:03<00:00, 3655.34ex/s]
Regenerating evaluation data.
  0%|          | 0/32109 [00:00<?, ?ex/s]Saving evaluation dataset to /Users/danielpham/Google Drive/ldh/output/eval
Encoding Test Data:
100%|██████████| 32109/32109 [00:11<00:00, 2704.23ex/s]


In [12]:
%autoreload 2 

from datetime import datetime
from pathlib import Path

import torch
import pandas as pd 
import pytorch_lightning as lit 

from pytorch_lightning.callbacks import EarlyStopping, ModelCheckpoint, Callback
from pl_bolts.callbacks import PrintTableMetricsCallback

save_dir = Path.cwd() / 'output'
logger = lit.loggers.TensorBoardLogger(save_dir=save_dir)

# TODO: Stop when the difference between the train loss starts to be greater than the val_loss
# ! Records the three models with the lowest monitored metric
class CVAccumulatorCallback(Callback):
    def validation_epoch_end(self, trainer):
      pass
    def test_epoch_end(self, trainer):
        print("Fit is ending!")

# Stops when the val loss stops changing
callback_earlystopping = EarlyStopping(monitor='val_loss', min_delta=0.0001, patience=5, verbose=True)
# Saves the top 3 checkpoints
callback_checkpoint = ModelCheckpoint(monitor="val_loss", mode='min', save_top_k=3,
        dirpath=save_dir,
        #filename='reappraisal-model-frozenbert-{epoch_02d}-{val_loss:.2f}-gpu1',
      )

# TODO: Refactor into main loop 
trainer = ReappTrainer(
    logger=logger,
    max_epochs=50,
    fast_dev_run=3,
    gradient_clip_val=1.0,
    progress_bar_refresh_rate=30,
    #gpus=1,
    terminate_on_nan=True,
    weights_summary="top",
    weights_save_path=save_dir,
    callbacks=[callback_earlystopping, callback_checkpoint]
  )

# TODO: eventual refactoring into main 
# def main(hparams):
#     model = LightningModule()
#     trainer = Trainer(gpus=hparams.gpus)
#     trainer.fit(model)

# Fit model
trainer.tune(model, ldhdata.train_dataloader())
trainer.fit(model, ldhdata.train_dataloader(), val_dataloaders=ldhdata.val_dataloader())
results = trainer.test(model, ldhdata.test_dataloader())
model

#5-fold CV
cv_results = []
for i in range(3):
  model = LightningReapp()
  train_data = ldhdata.get_train_dataloader(i)
  val_data = ldhdata.get_val_dataloader(i)
  trainer.fit(model, train_data, val_data)
  results = trainer.test(model, ldhdata.test_dataloader())
  cv_results.append(results)


EarlyStopping mode set to min for monitoring val_loss.


TypeError: __init__() missing 1 required positional argument: 'num_folds'

# Test Dataset

In [233]:
trainer.logged_metrics

{'train_loss': 16.61376953125, 'epoch': 0, 'val_loss': 9.040050506591797}

In [46]:
results = []
n = 0

for batch in ldhdata.test_dataloader():
  # TODO: at what point is "response" removed from the batch after passing it into forward? Why can't we just keep the response? 
  out = system(batch['input_ids'].cuda(), batch['attention_mask'].cuda())
  results.append((batch['addcode'], batch['daycode'], batch['Condition'], batch['response'], out.sum(dim=1).cpu()))
 v

In [47]:
# results = pd.Series(results).map(lambda row: (row[0], row[1], row[2].detach().numpy()))

results_df = pd.DataFrame.from_records(results)
results_blown = results_df.apply([pd.DataFrame.from_records, pd.DataFrame.transpose], axis=1)
new_df = results_blown.reset_index(drop=True)
new_df.columns = ['addcode', 'daycode', 'condition', 'response', 'score']
new_df['score'] = new_df['score'].transform(torch.Tensor.detach).apply(torch.Tensor.numpy)
new_df['daycode'] = new_df['daycode'].transform(torch.Tensor.detach).apply(torch.Tensor.numpy)
new_df


,addcode,daycode,condition,response,score
0,XAQ1001,1.0,RegNeg,this is a photo of a man and woman who look li...,5.895890
1,XAQ1001,1.0,RegNeg,"There was a couple arguing, with what looks li...",5.122256
2,XAQ1001,1.0,RegNeg,Someone's insides are cut open some time in th...,5.139289
3,XAQ1001,1.0,RegNeg,"A man is being held down by a police officer, ...",4.808964
4,XAQ1001,1.0,RegNeg,A police officer is arresting someone in the p...,4.837472
...,...,...,...,...,...
22967,YAS2019,1.0,RegNeg,"Not highly complex, just kinda a w.",3.375435
22968,YAS2019,1.0,RegNeg,Very gloomy background.,4.481534
22969,YAS2019,1.0,RegNeg,Looking into distance.,5.407123
22970,YAS2019,1.0,RegNeg,His ribs are visible.,6.140152
